<a href="https://colab.research.google.com/github/xiao-yucheng0625/ML-Progression-Journal/blob/main/%E4%BD%BF%E7%94%A8_DreamShaper_v8_%E5%89%B5%E9%80%A0%E5%A4%9A%E9%A2%A8%E6%A0%BC%E5%9C%96%E7%89%87%E7%9A%84_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 打造 Stable Diffusion 的 WebUI

### 1. 安裝必要套件

In [ ]:
!pip install diffusers transformers accelerate safetensors huggingface_hub gradio --upgrade

In [ ]:
# from google.colab import userdata

# hf_token = userdata.get("HuggingFace")
# login(token=hf_token)

In [ ]:
from diffusers import StableDiffusionPipeline, UniPCMultistepScheduler
import torch
import gc
import matplotlib.pyplot as plt
import gradio as gr
import random

### 2. 指定並讀入模型

In [ ]:
# 使用 DreamShaper v8 模型
model_name = "Lykon/dreamshaper-8"

注意有可能要停用 `use_safetensors=True`。

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda")

In [ ]:
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

In [ ]:
# 預設風格對應表
style_prompts = {
    "Realistic": {
        "enhance": "masterpiece, ultra high quality, realistic lighting",
        "negative": "blurry, bad anatomy, poorly drawn"
    },
    "Anime": {
        "enhance": "anime style, colorful, vivid, highly detailed",
        "negative": "bad anatomy, blurry, extra limbs"
    },
    "Fantasy": {
        "enhance": "fantasy art, ethereal, magical, dramatic lighting",
        "negative": "low detail, low resolution, blurry"
    }
}

In [ ]:
# 根據用戶選擇的風格，自動更新 enhance 與 negative prompt
def update_style(style):
    return style_prompts[style]['enhance'], style_prompts[style]['negative']

In [ ]:
scheduler_options = ["UniPCMultistepScheduler", "DDIMScheduler", "EulerAncestralDiscreteScheduler"]

In [ ]:
# 根據用戶選擇的排程器，切換 scheduler
def update_scheduler(selected_scheduler):
    if selected_scheduler == "UniPCMultistepScheduler":
        pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
    elif selected_scheduler == "DDIMScheduler":
        pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
    elif selected_scheduler == "EulerAncestralDiscreteScheduler":
        pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

### 3. 生成的函式

In [ ]:
def generate_images(prompt, use_enhance, enhance_text, use_negative, negative_text,
                    use_custom_seed, custom_seed, height, width, steps, num_images):

    height = int(height)
    width = int(width)

    if height % 8 != 0 or width % 8 != 0:
        raise ValueError("高度和寬度必須是8的倍數！")

    if use_custom_seed:
        base_seed = int(custom_seed)
    else:
        base_seed = random.randint(0, 2**32 - 1)

    seeds = [base_seed + i for i in range(num_images)]

    prompts = []
    negative_prompts = []
    generators = []

    final_prompt = prompt
    if use_enhance and enhance_text:
        final_prompt = prompt + ", " + enhance_text

    final_negative = negative_text if use_negative else None

    for seed in seeds:
        g = torch.Generator("cuda").manual_seed(seed)
        generators.append(g)
        prompts.append(final_prompt)
        negative_prompts.append(final_negative)

    gc.collect()
    torch.cuda.empty_cache()

    images = []
    for i in range(num_images):
        with torch.no_grad():
            image = pipe(
                prompt=prompts[i],
                negative_prompt=negative_prompts[i] if final_negative else None,
                height=height,
                width=width,
                num_inference_steps=steps,
                guidance_scale=7.5,
                generator=generators[i]
            ).images[0]
            images.append(image)

    return images, f"使用的 random seeds: {seeds}"

### 4. 打造 Gradio Web App

In [ ]:
with gr.Blocks(css=".gradio-container {background-color: #FAFAFA; padding: 20px;} .gr-button {font-size: 18px; background: linear-gradient(to right, #667eea, #764ba2); color: white;}") as demo:
    gr.Markdown("""
    # 🎨 DreamShaper v8 互動圖像生成器
    輸入提示詞、選擇風格與排程器，即刻生成夢幻風格作品！
    """)

    with gr.Row():
        with gr.Column(scale=6):
            style = gr.Dropdown(list(style_prompts.keys()), label="風格 Style", value="Realistic")
            scheduler_select = gr.Dropdown(scheduler_options, label="排程器 Scheduler", value="UniPCMultistepScheduler")
            prompt = gr.Textbox(label="Prompt", placeholder="請輸入你的提示詞 (prompt)", lines=3)
            with gr.Row():
                use_enhance = gr.Checkbox(label="加強 Prompt", value=True)
                enhance_text = gr.Textbox(label="加強內容")
            with gr.Row():
                use_negative = gr.Checkbox(label="使用 Negative Prompt", value=True)
                negative_text = gr.Textbox(label="Negative Prompt 內容")
            with gr.Row():
                use_custom_seed = gr.Checkbox(label="自訂 Random Seed", value=False)
                custom_seed = gr.Number(label="指定 seed (選填)", value=42)
            with gr.Row():
                height = gr.Dropdown(["512", "768", "1024"], label="高度 Height", value="512")
                width = gr.Dropdown(["512", "768", "1024"], label="寬度 Width", value="512")
            with gr.Row():
                steps = gr.Slider(10, 50, value=20, step=5, label="生成步數 (Steps)")
                num_images = gr.Slider(1, 4, step=1, value=1, label="生成張數")
            generate_btn = gr.Button("🚀 開始生成！")

        with gr.Column(scale=6):
            gallery = gr.Gallery(label="生成結果", columns=2, object_fit="contain", height="auto")
            seed_info = gr.Label(label="使用的 Random Seeds")

    style.change(fn=update_style, inputs=style, outputs=[enhance_text, negative_text])
    scheduler_select.change(fn=update_scheduler, inputs=scheduler_select, outputs=None)

    generate_btn.click(
        fn=generate_images,
        inputs=[prompt, use_enhance, enhance_text, use_negative, negative_text,
                use_custom_seed, custom_seed, height, width, steps, num_images],
        outputs=[gallery, seed_info]
    )

In [ ]:
demo.launch(share=True, debug=True)